In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from google_play_scraper import app, Sort, reviews_all, permissions

import threading
import time

import pickle

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-02scvt9r because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer,TFAutoModel, TFAutoModelWithLMHead
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

%env SENTENCE_TRANSFORMERS_HOME=./.config
%env TRANSFORMERS_CACHE=./transformers
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    if text is None or text == '':
        return ''
    
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)



MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
# !rm -r ./cardiffnlp
MODEL = '/tf/school/UofM_School/Capstone/tweeteval_new/roberta-base-rt-sentiment'#tf_model.h5'

tokenizer = AutoTokenizer.from_pretrained(MODEL, local_files_only=True)#'tf_model.h5')
# tokenizer = AutoTokenizer.from_pretrained('./local_model_directory/')
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
labels = ['negative','neutral','positive']

# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

env: SENTENCE_TRANSFORMERS_HOME=./.config
env: TRANSFORMERS_CACHE=./transformers


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at /tf/school/UofM_School/Capstone/tweeteval_new/roberta-base-rt-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
def encode_text(text):
#     if text is None or text == '':
#         return ''
#     text = preprocess(text)
#     if text is None or text =='':
#         return
    encoded_input = tokenizer(text, return_tensors='tf', padding=True, truncation=True, max_length=512)
    return encoded_input
    
def GetReviewSentiment(encoded_input):
    output = model(encoded_input)
#     display(output)
    scores = output[0][0].numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    return labels[ranking[0]]

# display(reviews_df.columns)

In [4]:
# tokenizer(['this is my test','this is another test to try'], return_tensors='tf', padding=True, truncation=True, max_length=512)

In [5]:
mypath = 'topReviews/'
sentimentPath = 'sentReviews/'



from os import listdir, path
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [6]:
## Practice with only one and let's see how it goes...
# temp_df = pd.read_pickle(mypath + onlyfiles[0])
# display(temp_df.sort_values(by='content', ascending =False))
# temp_df=temp_df[temp_df['content']=='None']
# temp_df =temp_df.sample(n=10)
# temp_df['encoded_input'] = temp_df['content'].apply(lambda x: encode_text(x))

# temp_df['content'] = temp_df['content'].apply(lambda x: 'NA' if x == None or x == 'None' else x)

# for text in temp_df['content'].values:
#     print(text)
#     encode_text(text)
    
# encoded_data = encode_text(list(temp_df['content'].values))
# # display(encoded_data)

# output = model(encoded_data)
# # display(output)
# scores = output[0].numpy()
# scores = softmax(scores, axis=1)
# display(scores)
# ranking = np.argsort(scores,axis=1)
# display('ranking')
# display(ranking)
# # ranking = ranking[::-1]
# ranking = np.flip(ranking, axis=1)
# sentiment = [labels[x[0]] for x in ranking]
# display(sentiment)
# display(pd.Series(sentiment))
# temp_df['sentiment'] = pd.Series(sentiment).values
# # numpy.flip(ranking, axis=1)
# # temp_df.append(pd.Series(scores))
# # # temp_df['sentiment'].append(scores)
# # display(temp_df.head())
# # # display("scores")
# # # display(scores.shape)
# # # display(scores)
# # temp_df['sentiment'] = temp_df['sentiment'].apply(lambda x: softmax(x))
# # temp_df['sentiment'] = temp_df['sentiment'].apply(lambda x: np.argsort(x))
# # temp_df['sentiment'] = temp_df['sentiment'].apply(lambda x: x[::-1])
# # temp_df['sentiment'] = temp_df['sentiment'].apply(lambda x: labels[x[0]])


# # scores = softmax(scores)
# # display("softmax")
# # display(scores)
# # ranking = np.argsort(scores)
# # display("sort")
# # display(scores)
# # ranking = ranking[::-1]
# # display(ranking)
# # ranking = ranking[::-1]
# # display(ranking)


# # temp_df['sentiment'] = GetReviewSentiment(encoded_data)

# # temp_df['sentiment'] = temp_df['encoded_input'].apply(lambda x: GetReviewSentiment(x))
# temp_df.head(10)

In [7]:
# temp_df.where(temp_df['content']=='None').apply(lambda x: 'NA' if x == 'None' else x)
# temp_df.head()

In [8]:
# display(pd.Series(sentiment))

# temp_df['sentiment'] = pd.Series(sentiment).values
# temp_df[['content','sentiment']].head(10)

In [38]:
def getSentiment(text):
        encoded_data = encode_text(list(text))
        output = model(encoded_data)
        scores = output[0].numpy()
        scores = softmax(scores, axis=1)
        ranking = np.argsort(scores,axis=1)
        ranking = np.flip(ranking, axis=1)
        sentiment = [labels[x[0]] for x in ranking]
        return sentiment

def batch_process_df(df, batch_size):
    
    
    timeEstimates = {}
    timeEstimates['count'] = 0
    timeEstimates['value'] = 0
    
    import time

#     start = time.time()
#     print("hello")
#     end = time.time()
#     print(end - start)

    numBatches = int(len(df)/batch_size)

    df['content'] = df['content'].apply(lambda x: 'NA' if x == None or x == 'None' else x)
    df['sentiment'] = df['content']
    
    final_df = None
    for i in range(0,numBatches):
        
        # show estimated time every 20 batches
        
        # we only want to show when the minute from last shown changes.
        remainingTime = int(round((timeEstimates['value'] * (numBatches-i))/60, 0))
        
#         changedMinute = int(round(timeEstimates['value']/60, 0)) != remainingTime
        if(timeEstimates['count'] > 0 and timeEstimates['count'] % 100 == 0):
            print(f"Estimated Time Remaining: {remainingTime} m")
        
        start = time.time()
        
        df_1 = df.iloc[i*batch_size: (i*batch_size) + batch_size,:]
        # call the sentiment analizer on our batch
        encoded_data = encode_text(list(df_1['content'].values))
        
        output = model(encoded_data)
        scores = output[0].numpy()
        scores = softmax(scores, axis=1)
        ranking = np.argsort(scores,axis=1)
        ranking = np.flip(ranking, axis=1)
        sentiment = [labels[x[0]] for x in ranking]
        
        # attach the sentiment data to the df we are using
        df_1['sentiment'] = pd.Series(sentiment).values
        
        
        #  merge together into one big df
        if final_df is None:
            final_df = df_1
        else:
            final_df = pd.concat([final_df, df_1],ignore_index=True)
            
        end = time.time()
        
        # let's average out the timer
        executionTime = end - start
        
#         print(end - start)
        
        if(timeEstimates['count']== 0):
            timeEstimates['count'] = 1
            timeEstimates['value'] = executionTime
        else:
            # undo the averaging we did and append this.
            oldTotal = timeEstimates['value'] * timeEstimates['count']
            
            timeEstimates['count'] = timeEstimates['count'] + 1
            timeEstimates['value'] = (oldTotal + executionTime)/timeEstimates['count']
        
            
    if len(df) % batch_size != 0:
        # we have a few remaining items to process
        df_1 = df.iloc[numBatches*batch_size:,:]
        # call the sentiment analizer on our batch
        encoded_data = encode_text(list(df_1['content'].values))
        
        output = model(encoded_data)
        scores = output[0].numpy()
        scores = softmax(scores, axis=1)
        ranking = np.argsort(scores,axis=1)
        ranking = np.flip(ranking, axis=1)
        sentiment = [labels[x[0]] for x in ranking]
        
        # attach the sentiment data to the df we are using
        df_1['sentiment'] = pd.Series(sentiment).values
        final_df = pd.concat([final_df, df_1],ignore_index=True)
#         The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
        
    return final_df

def WorkerJob(onlyfiles, threadNum):
    for app_id in onlyfiles:
        print(f"thread {threadNum}: loading file: {app_id}")
        temp_df = pd.read_pickle(mypath + app_id)
        
        print(f"thread {threadNum}: encoding text for analysis...")
#         temp_df['content'] = temp_df['content'].apply(lambda x: x if x !='None' or x != None else ' None ')
#         display(temp_df['content'].values[:100])
        temp_df['content'] = temp_df['content'].apply(lambda x: 'NA' if x == None or x == 'None' else x)
#         encoded_data = encode_text(list(temp_df['content'].values))
        
        print(f"thread {threadNum}: performing sentiment analysis on: {temp_df['title'].unique()[0]}")
        temp_df = batch_process_df(temp_df, 16)
#      Try using .loc[row_indexer,col_indexer] = value instead   
        
#         output = model(encoded_data)
#         scores = output[0].numpy()
#         scores = softmax(scores, axis=1)
#         ranking = np.argsort(scores,axis=1)
#         ranking = np.flip(ranking, axis=1)
#         sentiment = [labels[x[0]] for x in ranking]
#         temp_df['sentiment'] = pd.Series(sentiment).values
        
        print(f"thread {threadNum}: Done with {app_id}")
        temp_df.to_pickle(sentimentPath + app_id)
    

In [39]:
# display(reviews_df.columns)
# reviews_df.head(5)

# display(temp_df[temp_df['content']])
# display(temp_df['content'].sort_values(ascending=True).values)

In [40]:
WorkerJob(onlyfiles, 'only worker')

thread only worker: loading file: com.miHoYo.GenshinImpact.pkl
thread only worker: encoding text for analysis...
thread only worker: performing sentiment analysis on: Genshin Impact


<ipython-input-38-6e3985ef19b8>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['sentiment'] = pd.Series(sentiment).values


Estimated Time Remaining: 65 m
Estimated Time Remaining: 65 m
Estimated Time Remaining: 64 m
Estimated Time Remaining: 64 m
Estimated Time Remaining: 64 m
Estimated Time Remaining: 64 m
Estimated Time Remaining: 63 m
Estimated Time Remaining: 63 m
Estimated Time Remaining: 63 m
Estimated Time Remaining: 63 m
Estimated Time Remaining: 62 m
Estimated Time Remaining: 62 m
Estimated Time Remaining: 62 m
Estimated Time Remaining: 62 m
Estimated Time Remaining: 62 m
Estimated Time Remaining: 61 m
Estimated Time Remaining: 61 m
Estimated Time Remaining: 61 m
Estimated Time Remaining: 61 m
Estimated Time Remaining: 60 m
Estimated Time Remaining: 60 m
Estimated Time Remaining: 60 m
Estimated Time Remaining: 60 m
Estimated Time Remaining: 59 m
Estimated Time Remaining: 59 m
Estimated Time Remaining: 59 m
Estimated Time Remaining: 59 m
Estimated Time Remaining: 59 m
Estimated Time Remaining: 58 m
Estimated Time Remaining: 58 m
Estimated Time Remaining: 58 m
Estimated Time Remaining: 58 m
Estimate

<ipython-input-38-6e3985ef19b8>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['sentiment'] = pd.Series(sentiment).values


thread only worker: Done with com.miHoYo.GenshinImpact.pkl
thread only worker: loading file: com.bandainamcogames.dbzdokkanww.pkl
thread only worker: encoding text for analysis...
thread only worker: performing sentiment analysis on: DRAGON BALL Z DOKKAN BATTLE


...

<ipython-input-38-6e3985ef19b8>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['sentiment'] = pd.Series(sentiment).values


thread only worker: Done with com.netmarble.mherosgb.pkl
thread only worker: loading file: com.bandainamcoent.hiroacawwus.pkl
thread only worker: encoding text for analysis...
thread only worker: performing sentiment analysis on: MY HERO ULTRA IMPACT


<ipython-input-38-6e3985ef19b8>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['sentiment'] = pd.Series(sentiment).values


Estimated Time Remaining: 0 m
thread only worker: Done with com.bandainamcoent.hiroacawwus.pkl


<ipython-input-38-6e3985ef19b8>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['sentiment'] = pd.Series(sentiment).values


In [ ]:
# countSplit =(int)(len(onlyfiles) / 4)

# print( countSplit)
# # desiredTickers.ticker.unique()[2492]
# tickersArray=[]
# mylist=onlyfiles
# tickersArray.append(mylist[:countSplit*1])
# # print(desiredTickers.ticker.unique()[countSplit])
# tickersArray.append(mylist[countSplit*1:countSplit*2])
# tickersArray.append(mylist[countSplit*2:countSplit*3])
# tickersArray.append(mylist[countSplit*3:])

# tickers0 = tickersArray[0]
# tickers1 = tickersArray[1]
# tickers2 = tickersArray[2]
# tickers3 = tickersArray[3]

# yearsData = 9
# def worker0():
#     WorkerJob(tickers0, threading.current_thread().getName())

# def worker1():
#     WorkerJob(tickers1, threading.current_thread().getName())

# def worker2():
#     WorkerJob(tickers2, threading.current_thread().getName())

# def worker3():
#     WorkerJob(tickers3, threading.current_thread().getName())


# # t = threading.Thread(name='my_service', target=my_service)
# w1 = threading.Thread(name='worker0', target=worker0)
# w2 = threading.Thread(name='worker1', target=worker1)
# w3 = threading.Thread(name='worker2', target=worker2)
# w4 = threading.Thread(name='worker3', target=worker3)
# #     w2 = threading.Thread(target=worker)  # use default name

# w1.start()
# w2.start()
# w3.start()
# w4.start()

In [ ]:

# display(reviews_df.dtypes)

In [ ]:
## Checkout all the apps we have

In [ ]:
# TODO: use a topic model to rate positive/neutral/negative and see how that score changes over time.
# !pip install transformers
# !pip install transformers
# !pip install torch

# %env SENTENCE_TRANSFORMERS_HOME=./.config
# # %env TRANSFORMERS_CACHE=./transformers
# # %env PYTORCH_TRANSFORMERS_CACHE=./.config
# # %env PYTORCH_PRETRAINED_BERT_CACHE=./.config
# %env HF_DATASETS_OFFLINE=1 
# %env TRANSFORMERS_OFFLINE=1
# !wget 'https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/config.json'
# import sys

# import os
# import os.path
# os.environ["SENTENCE_TRANSFORMERS_HOME"] = "./.config"
# # sys.path.append("./.config")
# # !wget 'https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/tf_model.h5'
# !printenv

In [ ]:
# !cat 'config.json'
# !wget 'https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/special_tokens_map.json'
# !wget 'https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/vocab.json'
# !cat 'tweeteval_new/roberta-base-rt-sentiment/config.json.1' > 'tweeteval_new/roberta-base-rt-sentiment/config.json'

# !cat 'tweeteval_new/roberta-base-rt-sentiment/vocab.json.1' > 'tweeteval_new/roberta-base-rt-sentiment/vocab.json'
# !mkdir 'tweeteval_new'
# !mkdir 'tweeteval_new/roberta-base-rt-sentiment/'

In [ ]:
# # from transformers import AutoModelForSequenceClassification
# from transformers import TFAutoModelForSequenceClassification
# from transformers import AutoTokenizer,TFAutoModel, TFAutoModelWithLMHead
# import numpy as np
# from scipy.special import softmax
# import csv
# import urllib.request

# %env SENTENCE_TRANSFORMERS_HOME=./.config
# %env TRANSFORMERS_CACHE=./transformers
# # Preprocess text (username and link placeholders)
# def preprocess(text):
#     new_text = []
#     if text is None or text == '':
#         return ''
    
#     for t in text.split(" "):
#         t = '@user' if t.startswith('@') and len(t) > 1 else t
#         t = 'http' if t.startswith('http') else t
#         new_text.append(t)
#     return " ".join(new_text)



# task='sentiment'
# MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
# # !rm -r ./cardiffnlp
# MODEL = '/tf/school/UofM_School/Capstone/tweeteval_new/roberta-base-rt-sentiment'#tf_model.h5'

# tokenizer = AutoTokenizer.from_pretrained(MODEL, local_files_only=True)#'tf_model.h5')
# # tokenizer = AutoTokenizer.from_pretrained('./local_model_directory/')
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# labels = ['negative','neutral','positive']

# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

In [ ]:
# # Tasks:
# # emoji, emotion, hate, irony, offensive, sentiment
# # stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

# task='sentiment'
# MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
# # !rm -r ./cardiffnlp
# MODEL = '/tf/school/UofM_School/Capstone/tweeteval_new/roberta-base-rt-sentiment'#tf_model.h5'

# tokenizer = AutoTokenizer.from_pretrained(MODEL, local_files_only=True)#'tf_model.h5')
# # tokenizer = AutoTokenizer.from_pretrained('./local_model_directory/')
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)


# # https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment
# # https://github.com/cardiffnlp/tweeteval/blob/main/datasets/sentiment/mapping.txt
# labels = ['negative','neutral','positive']
# # # download label mapping
# # labels=[]
# # mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt"
# # with urllib.request.urlopen(mapping_link) as f:
# #     html = f.read().decode('utf-8').split("\n")
# #     csvreader = csv.reader(html, delimiter='\t')
# # labels = [row[1] for row in csvreader if len(row) > 1]


# # # PT
# # model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# # model.save_pretrained(MODEL)
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)  # adding this line

# text = "Good night 😊"
# # text = "hate it"
# text = preprocess(text)

# #return tensors:
# # 'tf': Return TensorFlow tf.constant objects.
# # 'pt': Return PyTorch torch.Tensor objects.
# # 'np': Return Numpy np.ndarray objects.
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(**encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

# # # TF
# # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# # model.save_pretrained(MODEL)

# # text = "Good night 😊"
# # encoded_input = tokenizer(text, return_tensors='tf')
# # output = model(encoded_input)
# # scores = output[0][0].numpy()
# # scores = softmax(scores)

# ranking = np.argsort(scores)
# ranking = ranking[::-1]
# for i in range(scores.shape[0]):
#     l = labels[ranking[i]]
#     s = scores[ranking[i]]
#     print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
# labels[ranking[0]]

In [ ]:
# print(scores.shape)

## Getting back to review data

In [ ]:
# def GetReviewSentiment(text):
#     if text is None or text == '':
#         return ''
#     text = preprocess(text)
#     if text is None or text =='':
#         return
#     encoded_input = tokenizer(text, return_tensors='tf')
#     output = model(**encoded_input)
#     scores = output[0][0].numpy()
#     scores = softmax(scores)

#     ranking = np.argsort(scores)
#     ranking = ranking[::-1]
#     return labels[ranking[0]]

# display(reviews_df.columns)
# reviews_df['sentiment'] = reviews_df['content'].apply(lambda x: GetReviewSentiment(x))

In [ ]:
# reviews_df.head()

In [42]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 325 kB 664 kB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 134 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 26.7 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 801 kB/s eta 0:00:01
     |████████████████████████████████| 308 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 1.5 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer

In [ ]:
## Let's try summarizing:
# !pip install datasets

import pandas as pd
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm


model = "bart-large-cnn"
# summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY")
summarizer = pipeline("summarization", model=model, local_files_only=True)


df = pd.read_pickle('sentReviews/com.miHoYo.GenshinImpact.pkl')

In [7]:
# df_dataset = Dataset.from_pandas(df)
from datasets import load_dataset, load_metric, Audio


summarizer(' '.join(df['content'].values[0:100]))

# # pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h", device=0)
# # dataset = datasets.load_dataset("superb", name="asr", split="test")

# # KeyDataset (only *pt*) will simply return the item in the dict returned by the dataset item
# # as we're not interested in the *target* part of the dataset.
# for out in tqdm(summarizer(KeyDataset(dataset, "file"))):
#     print(out)
#     break
#     # {"text": "NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD NIGHT HUSBAND"}
#     # {"text": ....}
#     # ....

[{'summary_text': "The game is too good to be free... one of the BEST rpg games I've ever played... keep up the great work!! After this recent fiasco I can't support this game anymore, tons of unfixed bugs that hinder the player while fun glitches are fixed immediatly. I will rate you a 5 star if you give me Ei. A great Game and super good lore but the creators of this game is so gready that even can make the whole world turn against them. The game has insane graphics best RPG game ever."}]

In [4]:
df['content'].values[0]

"𝙿𝚕𝚊𝚢𝚎𝚍 𝚝𝚑𝚒𝚜 𝚐𝚊𝚖𝚎 𝚊𝚋𝚘𝚞𝚝 5 𝚖𝚘𝚗𝚝𝚑𝚜 𝚊𝚐𝚘 𝚋𝚞𝚝 𝚖𝚢 𝚏𝚛𝚒𝚎𝚗𝚍 𝚍𝚎𝚕𝚎𝚝𝚎𝚍 𝚒𝚝 𝚊𝚗𝚍 𝙸 𝚓𝚞𝚜𝚝 𝚍𝚘𝚠𝚗𝚕𝚘𝚊𝚍𝚎𝚍 𝚊𝚐𝚊𝚒𝚗 𝙸 𝚍𝚒𝚍𝚗'𝚝 𝚎𝚊𝚛𝚕𝚒𝚎𝚛 𝙲𝚞𝚣 𝙸 𝚑𝚊𝚟𝚎 𝚜𝚌𝚑𝚘𝚘𝚕 𝚋𝚞𝚝 𝚗𝚘𝚠 𝙸 𝚑𝚊𝚟𝚎 𝚖𝚘𝚛𝚎 𝚝𝚒𝚖𝚎 𝚘𝚗 𝚖𝚢 𝚑𝚊𝚗𝚍𝚜 𝚊𝚗𝚍 𝚐𝚊𝚗𝚗𝚊 𝚙𝚕𝚊𝚢 𝚒𝚝 𝚊𝚜 𝚖𝚞𝚌𝚑 𝙰𝚜 𝙸 𝚠𝚊𝚗𝚝𝚎𝚍 𝚝𝚘 𝚋𝚊𝚌𝚔 𝚝𝚑𝚎𝚗"

In [15]:
# display('\n'.join(df['content'].values[0:100]))
for rev in df['content'].values[0:100]:
    display(rev)

"𝙿𝚕𝚊𝚢𝚎𝚍 𝚝𝚑𝚒𝚜 𝚐𝚊𝚖𝚎 𝚊𝚋𝚘𝚞𝚝 5 𝚖𝚘𝚗𝚝𝚑𝚜 𝚊𝚐𝚘 𝚋𝚞𝚝 𝚖𝚢 𝚏𝚛𝚒𝚎𝚗𝚍 𝚍𝚎𝚕𝚎𝚝𝚎𝚍 𝚒𝚝 𝚊𝚗𝚍 𝙸 𝚓𝚞𝚜𝚝 𝚍𝚘𝚠𝚗𝚕𝚘𝚊𝚍𝚎𝚍 𝚊𝚐𝚊𝚒𝚗 𝙸 𝚍𝚒𝚍𝚗'𝚝 𝚎𝚊𝚛𝚕𝚒𝚎𝚛 𝙲𝚞𝚣 𝙸 𝚑𝚊𝚟𝚎 𝚜𝚌𝚑𝚘𝚘𝚕 𝚋𝚞𝚝 𝚗𝚘𝚠 𝙸 𝚑𝚊𝚟𝚎 𝚖𝚘𝚛𝚎 𝚝𝚒𝚖𝚎 𝚘𝚗 𝚖𝚢 𝚑𝚊𝚗𝚍𝚜 𝚊𝚗𝚍 𝚐𝚊𝚗𝚗𝚊 𝚙𝚕𝚊𝚢 𝚒𝚝 𝚊𝚜 𝚖𝚞𝚌𝚑 𝙰𝚜 𝙸 𝚠𝚊𝚗𝚝𝚎𝚍 𝚝𝚘 𝚋𝚊𝚌𝚔 𝚝𝚑𝚎𝚗"

'Best game I have ever play, AFTER I PLAYED THIS GAME I FINALLY GOT A GIRL, CAUSE NOW IM AWESOME👌'

'I will rate you a 5 star if you give me Ei.'

'I like it'

'A great Game and super good lore but the creators of this game is so gready that even can make the whole world turn against them'

...

'Genshin ilysm km sgt baik hati kan berikan sy yoimiya, ayaka donk pwis 😥😥😥 <3333 btw makasih kokominya di pity 8‼️'

"Yo i got raiden's weapon in 20 pity"

"Mihoyo let me a win 50/50 on ayato's banner"

"Sometimes I wonder if I'm even playing the same game as my friends. They keep winning 50/50 and at really low pull counts while I'm stuck with hard pity and losing 50/50. Even the weapon banner thats absolute trash lets them win. Is this a matter of account or I'm just a really unlucky human being."

'good'

In [16]:
## Try summary on positive only data
temp_df = df[df['sentiment'] == 'positive']
summarizer(' '.join(temp_df['content'].values[0:100]))

[{'summary_text': "Gensin is probably the best open world game ever recommended to me. The animation nice, characters power purfect, sujet amazing, I wish play again in this ! :3 (For parents : It's for kids and there not censored or blood) The gane is gorgeous and so much fun! Good game ngl It's fun to play and explore the world and I haven't really experienced problems."}]

In [19]:

temp_df = df[df['sentiment'] == 'negative']
summarizer(' '.join(temp_df['content'].values[0:100]))

[{'summary_text': "I love this game but when there's no event, i feel so bored to play genshin, and worst part there is no content to play beside artifact grind. Lack of synergy and power when compared to other elements makes it almost useless in the endgame bosses/content we have now. Good lore, nice character designs and cool open world but... you can easily get burned out. Game is fun but a huge lack of communication with the community still exists."}]

In [18]:

temp_df = df[df['sentiment'] == 'neutral']
summarizer(' '.join(temp_df['content'].values[0:100]))

[{'summary_text': "The game is fun and all but ever time I try to play on my android phone I have to wait 1h before beginning and some times it stops loading. Plz fix this issue so I can play. I was building Ayaka and out of 9 artifacts, only 2 were blizzard strayer with Def% and healing bonus stats. It could have been 5 stars if you could summon a 5 star at the beginning as I have been playing for weeks and I don't even have one 5 star Grape. Please make an option where u can have a higher chance in getting an artifact of choice when we do the domain alone. The game is not a game, it's a tool."}]